In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras import metrics


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
dftrain = pd.read_csv(train_file_path, sep="\t", header=None, names=['label', 'text'])
dftest = pd.read_csv(test_file_path, sep="\t", header=None, names=['label', 'text'])



In [ ]:
# Preprocess data
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(dftrain['text'])
X_train = sequence.pad_sequences(tokenizer.texts_to_sequences(dftrain['text']), maxlen=100)
X_test = sequence.pad_sequences(tokenizer.texts_to_sequences(dftest['text']), maxlen=100)
le = LabelEncoder()
y_train = le.fit_transform(dftrain['label'])
y_test = le.transform(dftest['label'])



In [ ]:
# Build model
model = Sequential([
    Embedding(1000, 128),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
# Train model
model.fit(X_train, y_train, batch_size=32, epochs=2, validation_split=0.3)



In [ ]:
# Evaluate model
_, accuracy = model.evaluate(X_test, y_test)
print(f"Testing set accuracy: {accuracy:.2f}")



In [ ]:
# Function to predict messages
def predict_message(pred_text):
    input_sequence = sequence.pad_sequences(tokenizer.texts_to_sequences([pred_text]), maxlen=100)
    pred = model.predict(input_sequence)[0][0]
    label = "spam" if pred >= 0.5 else "ham"
    return [pred, label]



In [ ]:
# Test function
def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"]
    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = all(predict_message(msg)[1] == ans for msg, ans in zip(test_messages, test_answers))
    print("You passed the challenge. Great job!" if passed else "You haven't passed yet. Keep trying.")

test_predictions()